# College Classification

    
The task is to use a college dataset to try to classify colleges as Private or Public based features:

    Private A factor with levels No and Yes indicating private or public university
    Apps Number of applications received
    Accept Number of applications accepted
    Enroll Number of new students enrolled
    Top10perc Pct. new students from top 10% of H.S. class
    Top25perc Pct. new students from top 25% of H.S. class
    F.Undergrad Number of fulltime undergraduates
    P.Undergrad Number of parttime undergraduates
    Outstate Out-of-state tuition
    Room.Board Room and board costs
    Books Estimated book costs
    Personal Estimated personal spending
    PhD Pct. of faculty with Ph.D.’s
    Terminal Pct. of faculty with terminal degree
    S.F.Ratio Student/faculty ratio
    perc.alumni Pct. alumni who donate
    Expend Instructional expenditure per student
    Grad.Rate Graduation rate
    
Testing 3 different tree methods:

* A single decision tree
* A random forest
* A gradient boosted tree classifier

In [1]:
import findspark
findspark.init('/home/matt/spark-3.1.1-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('college').getOrCreate()

In [3]:
# Load training data
df = spark.read.csv('College.csv',inferSchema=True,header=True)

In [4]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [5]:
df.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|
|      Adrian College|    Yes|1428|  1097|   336|       22|       50|       1036|         99|  

## Feature engineering

In [14]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [15]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [16]:
assembler = VectorAssembler(
  inputCols=['Apps',
             'Accept',
             'Enroll',
             'Top10perc',
             'Top25perc',
             'F_Undergrad',
             'P_Undergrad',
             'Outstate',
             'Room_Board',
             'Books',
             'Personal',
             'PhD',
             'Terminal',
             'S_F_Ratio',
             'perc_alumni',
             'Expend',
             'Grad_Rate'],
              outputCol="features")

In [17]:
output = assembler.transform(df)

In [18]:
# Convert Private column (the labels) into a categorical variable
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")
indexed = indexer.fit(output).transform(output)

In [19]:
final_data = indexed.select("features",'PrivateIndex')

In [20]:
#split data
train_data,test_data = final_data.randomSplit([0.7,0.3])

## Train model(s) using pipelines

In [21]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

Create all three models:

In [30]:
# Using defaults to make fair comparison
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features',numTrees=75)
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features',maxIter=150)

Train all three models:

In [31]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

## Model Comparison

In [32]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

## Evaluate models

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="accuracy")

In [35]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [37]:
print("Here are the results!")
print('-'*80)
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*80)
print('A ensemble using GBT had an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
--------------------------------------------------------------------------------
A single decision tree had an accuracy of: 89.42%
--------------------------------------------------------------------------------
A random forest ensemble had an accuracy of: 90.88%
--------------------------------------------------------------------------------
A ensemble using GBT had an accuracy of: 90.51%


In [48]:
# check models to see if any feature is a good predictor of college type
dtc_model.featureImportances

SparseVector(17, {0: 0.0184, 3: 0.0128, 5: 0.5257, 6: 0.0162, 7: 0.3318, 8: 0.0146, 9: 0.0103, 10: 0.0101, 11: 0.0242, 14: 0.036})

In [44]:
rfc_model.featureImportances

SparseVector(17, {0: 0.0357, 1: 0.061, 2: 0.1063, 3: 0.0168, 4: 0.0095, 5: 0.2206, 6: 0.106, 7: 0.1991, 8: 0.0594, 9: 0.006, 10: 0.0099, 11: 0.0217, 12: 0.0128, 13: 0.0501, 14: 0.0173, 15: 0.0366, 16: 0.0311})

In [45]:
gbt_model.featureImportances

SparseVector(17, {0: 0.0329, 1: 0.0065, 2: 0.0398, 3: 0.0166, 4: 0.0157, 5: 0.4137, 6: 0.0758, 7: 0.2719, 8: 0.0176, 9: 0.0033, 10: 0.0078, 11: 0.0341, 12: 0.024, 13: 0.0055, 14: 0.0223, 15: 0.0107, 16: 0.0018})

In [49]:
# Number of new students enrolled (feature 5, Enroll) has highest significance across models